In [1]:
import os
from tqdm.notebook import tqdm
from pathlib import Path
import pandas as pd
from nltk.lm import Vocabulary
import sys
sys.path.append("../../lib")
from metrics import levenshtein
import pickle

In [2]:
folder = "../../data/ICDAR2019_POCR_competition_dataset/ICDAR2019_POCR_competition_training_18M_without_Finnish/BG/"

In [3]:
output_folder = Path("../../data/bg")

In [4]:
files = sorted(os.listdir(folder))
len(files)

1

In [5]:
import glob

files = glob.glob(folder + '/**/*.txt', recursive=True)
len(files)

149

In [6]:
from multiprocessing import Pool

def extract(name):
    with open(name) as file:
        return file.readlines()
    
def create_windows(x):
    A, B, window_length = x
    assert len(A) == len(B)
    return [(A[i:i + window_length], B[i:i + window_length]) 
            for i in range(len(A) + 1)]
    
p = Pool(4)
    
data = list(p.imap_unordered(extract, tqdm(files), chunksize = 128))
len(data)

  0%|          | 0/149 [00:00<?, ?it/s]

149

In [7]:
# data = []
# for f in tqdm(files):
#     with open(f) as file:
#         data.append(file.readlines())

data = pd.DataFrame(data, 
                    columns = ["ocr_to_input", 
                               "ocr_aligned", 
                               "gs_aligned"])\
.assign(ocr_to_input = lambda df: df.ocr_to_input.str.replace("[OCR_toInput] ", "", regex = False),
        ocr_aligned = lambda df: df.ocr_aligned.str.replace("[OCR_aligned] ", "", regex = False),
        gs_aligned = lambda df: df.gs_aligned.str.replace("[ GS_aligned] ", "", regex = False))

print(data.shape)
data.head()

(149, 3)


,ocr_to_input,ocr_aligned,gs_aligned
0,"— 211 — Отъ миогочислеини наблюдения Паркет, е...","@— 211 — Отъ миогочислеини наблюдения Паркет, ...",211 Отъ многочисленни паблюдения Парк@съ ...
1,КониетТ; нмъ вТюлето бойно Презъ огиь се припк...,КониетТ; @нмъ вТюлето бойно Презъ огиь се прип...,337 Но предприемчивата злоба В'сь@рдцето ...
2,— 272 — казалъ азъ.—Така. Па каго чрВзъ насъ с...,@— 272 — казалъ азъ.—Така. Па каго чрВзъ насъ ...,272 казалъ азъ. Така. Но каго чрѣзъ насъ ...
3,",1 о 0. тО о диЯ внучета. Останаха живи само д...",",1 о 0. тО о диЯ внучета. Останаха живи само д...",@@ @ @ 433 двѣ внучета. Останахѫ живи само д...
4,— 91 — Какво ли има пакъ? Когато тичахъ нрФзъ ...,@— 91@@ — Какво ли има пакъ? Когато тичахъ нрФ...,91 Какво ли има акъ? Когато тичахъ прѣ...


In [8]:
data.applymap(len).describe()

,ocr_to_input,ocr_aligned,gs_aligned
count,149.000000,149.000000,149.000000
mean,1866.879195,1925.067114,1925.067114
std,628.106807,631.422736,631.422736
min,41.000000,41.000000,41.000000
25%,1205.000000,1339.000000,1339.000000
50%,2130.000000,2184.000000,2184.000000
75%,2344.000000,2391.000000,2391.000000
max,2771.000000,2969.000000,2969.000000


In [9]:
levenshtein(reference = data.gs_aligned.str.replace("@", ""), 
            hypothesis = data.ocr_to_input).cer.describe()

count    149.000000
mean      17.706168
std       18.898940
min        5.244338
25%        7.995029
50%        9.142607
75%       13.867277
max      104.950495
Name: cer, dtype: float64

In [10]:
levenshtein(reference = data.gs_aligned, 
            hypothesis = data.ocr_aligned).cer.describe()

count    149.000000
mean      16.136107
std       15.694098
min        5.233941
25%        7.915194
50%        9.055728
75%       13.478648
max       86.885246
Name: cer, dtype: float64

In [13]:
vocabulary = Vocabulary(data.ocr_to_input.sum() + data.ocr_aligned.sum() + data.gs_aligned.sum())
print(len(vocabulary))
with open(output_folder/"data/vocabulary.pkl", "wb") as file:
    pickle.dump(vocabulary, file)

188


In [14]:
dev = data.sample(n = 5, random_state = 1)
dev.to_pickle(output_folder/"data/dev.pkl")
dev.shape

(5, 3)

In [15]:
train = data.drop(dev.index)
train.to_pickle(output_folder/"data/train.pkl")
train.shape

(144, 3)

In [16]:
train.applymap(len).describe()

,ocr_to_input,ocr_aligned,gs_aligned
count,144.000000,144.000000,144.000000
mean,1868.173611,1924.298611,1924.298611
std,628.187473,631.285791,631.285791
min,41.000000,41.000000,41.000000
25%,1203.750000,1325.250000,1325.250000
50%,2128.000000,2184.500000,2184.500000
75%,2344.500000,2388.750000,2388.750000
max,2771.000000,2969.000000,2969.000000


In [17]:
dev.applymap(len).describe()

,ocr_to_input,ocr_aligned,gs_aligned
count,5.000000,5.000000,5.000000
mean,1829.600000,1947.200000,1947.200000
std,698.374756,709.870904,709.870904
min,767.000000,924.000000,924.000000
25%,1524.000000,1531.000000,1531.000000
50%,2140.000000,2163.000000,2163.000000
75%,2168.000000,2546.000000,2546.000000
max,2549.000000,2572.000000,2572.000000


In [18]:
levenshtein(reference = dev.gs_aligned.str.replace("@", ""), 
            hypothesis = dev.ocr_to_input).cer.describe()

count     5.000000
mean     22.770819
std      21.135585
min       6.233596
25%       6.947697
50%       8.899297
75%      44.914864
max      46.858639
Name: cer, dtype: float64

In [19]:
levenshtein(reference = dev.gs_aligned, 
            hypothesis = dev.ocr_to_input).cer.describe()

count     5.000000
mean     19.797821
std      17.135335
min       6.205095
25%       6.920684
50%       8.784096
75%      38.334643
max      38.744589
Name: cer, dtype: float64

In [20]:
window_length = 100

In [21]:
df = train#.head(100)
train_aligned = list(p.imap_unordered(create_windows, 
                                      tqdm(zip(df.ocr_aligned, 
                                               df.gs_aligned, 
                                               [window_length for x in df.ocr_aligned]), 
                                           total = len(df.ocr_aligned)),
                                      chunksize = 128))
s = []
for r in tqdm(train_aligned):
    s.extend(r)
train_aligned = pd.DataFrame(s, columns = ["source", "target"])
print(train_aligned.shape)
train_aligned.head()

  0%|          | 0/144 [00:00<?, ?it/s]

  0%|          | 0/144 [00:00<?, ?it/s]

(277243, 2)


,source,target
0,". — 32 — като Охридски епископи, че вь негово ...","@ 32 като Охридски е иско ъ, че въ негово ..."
1,"— 32 — като Охридски епископи, че вь негово и...","32 като Охридски е иско ъ, че въ негово и..."
2,"— 32 — като Охридски епископи, че вь негово им...","32 като Охридски е иско ъ, че въ негово им..."
3,"32 — като Охридски епископи, че вь негово име...","32 като Охридски е иско ъ, че въ негово име..."
4,"32 — като Охридски епископи, че вь негово име ...","32 като Охридски е иско ъ, че въ негово име ..."


In [22]:
train_aligned = train_aligned.assign(source = lambda df: df.source.str.replace("@", ""))
train_aligned.head()

,source,target
0,". — 32 — като Охридски епископи, че вь негово ...","@ 32 като Охридски е иско ъ, че въ негово ..."
1,"— 32 — като Охридски епископи, че вь негово и...","32 като Охридски е иско ъ, че въ негово и..."
2,"— 32 — като Охридски епископи, че вь негово им...","32 като Охридски е иско ъ, че въ негово им..."
3,"32 — като Охридски епископи, че вь негово име...","32 като Охридски е иско ъ, че въ негово име..."
4,"32 — като Охридски епископи, че вь негово име ...","32 като Охридски е иско ъ, че въ негово име ..."


In [23]:
dev_aligned = dev.apply(lambda r: create_windows((r["ocr_aligned"], r["gs_aligned"], window_length)), 
                            axis = 1).sum()
dev_aligned = pd.DataFrame(dev_aligned, columns = ["source", "target"])
print(dev_aligned.shape)
dev_aligned.head()

(9741, 2)


,source,target
0,— 290 — съ нАкакъвъ болезнена. блАсъкъ. Тя пог...,@@@@@@@ съ нѣкакъвъ болѣзнен@ъ блѣсъкъ. Тя пог...
1,290 — съ нАкакъвъ болезнена. блАсъкъ. Тя погл...,@@@@@@ съ нѣкакъвъ болѣзнен@ъ блѣсъкъ. Тя погл...
2,290 — съ нАкакъвъ болезнена. блАсъкъ. Тя погле...,@@@@@ съ нѣкакъвъ болѣзнен@ъ блѣсъкъ. Тя погле...
3,90 — съ нАкакъвъ болезнена. блАсъкъ. Тя поглед...,@@@@ съ нѣкакъвъ болѣзнен@ъ блѣсъкъ. Тя поглед...
4,0 — съ нАкакъвъ болезнена. блАсъкъ. Тя погледн...,@@@ съ нѣкакъвъ болѣзнен@ъ блѣсъкъ. Тя погледн...


In [24]:
dev_aligned = dev_aligned.assign(source = lambda df: df.source.str.replace("@", ""))
dev_aligned.head()

,source,target
0,— 290 — съ нАкакъвъ болезнена. блАсъкъ. Тя пог...,@@@@@@@ съ нѣкакъвъ болѣзнен@ъ блѣсъкъ. Тя пог...
1,290 — съ нАкакъвъ болезнена. блАсъкъ. Тя погл...,@@@@@@ съ нѣкакъвъ болѣзнен@ъ блѣсъкъ. Тя погл...
2,290 — съ нАкакъвъ болезнена. блАсъкъ. Тя погле...,@@@@@ съ нѣкакъвъ болѣзнен@ъ блѣсъкъ. Тя погле...
3,90 — съ нАкакъвъ болезнена. блАсъкъ. Тя поглед...,@@@@ съ нѣкакъвъ болѣзнен@ъ блѣсъкъ. Тя поглед...
4,0 — съ нАкакъвъ болезнена. блАсъкъ. Тя погледн...,@@@ съ нѣкакъвъ болѣзнен@ъ блѣсъкъ. Тя погледн...


In [25]:
train_aligned.to_pickle(output_folder/"data/train_aligned.pkl")
dev_aligned.to_pickle(output_folder/"data/dev_aligned.pkl")